In [1]:
import nltk

In [2]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [3]:
import pandas as pd

In [4]:
fake = pd.read_csv("Fake.csv")
true = pd.read_csv("True.csv")

In [5]:
display(fake.head())
display(true.head())

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [6]:
display(fake.subject.value_counts())

News               9050
politics           6841
left-news          4459
Government News    1570
US_News             783
Middle-east         778
Name: subject, dtype: int64

In [7]:
display(true.subject.value_counts())

politicsNews    11272
worldnews       10145
Name: subject, dtype: int64

In [8]:
fake["target"] = 0
true["target"] = 1

In [9]:
data = pd.concat([fake,true],axis=0)
data = data.reset_index(drop = True)
data = data.drop(['subject','date','title'],axis=1)
print(data.columns)

Index(['text', 'target'], dtype='object')


### TOKENIZATION

In [10]:
from nltk.tokenize import word_tokenize

In [11]:
data['text']=data['text'].apply(word_tokenize)

### STEMMING

In [12]:
print(data.head())

                                                text  target
0  [Donald, Trump, just, couldn, t, wish, all, Am...       0
1  [House, Intelligence, Committee, Chairman, Dev...       0
2  [On, Friday, ,, it, was, revealed, that, forme...       0
3  [On, Christmas, day, ,, Donald, Trump, announc...       0
4  [Pope, Francis, used, his, annual, Christmas, ...       0


In [13]:
from nltk.stem.snowball import SnowballStemmer
porter = SnowballStemmer("english")

In [14]:
def stem_it(text):
    return[porter.stem(word) for word in text]

In [15]:
data['text'] = data['text'].apply(stem_it)
print(data.head())

                                                text  target
0  [donald, trump, just, couldn, t, wish, all, am...       0
1  [hous, intellig, committe, chairman, devin, nu...       0
2  [on, friday, ,, it, was, reveal, that, former,...       0
3  [on, christma, day, ,, donald, trump, announc,...       0
4  [pope, franci, use, his, annual, christma, day...       0


### STOPWORD REMOVAL

In [16]:
def stop_it(t):
    dt=[word for word in t if len(word)>2]
    return dt

In [17]:
data['text']=data['text'].apply(stop_it)
print(data.head(10))

                                                text  target
0  [donald, trump, just, couldn, wish, all, ameri...       0
1  [hous, intellig, committe, chairman, devin, nu...       0
2  [friday, was, reveal, that, former, milwauke, ...       0
3  [christma, day, donald, trump, announc, that, ...       0
4  [pope, franci, use, his, annual, christma, day...       0
5  [the, number, case, cop, brutal, and, kill, pe...       0
6  [donald, trump, spent, good, portion, his, day...       0
7  [the, wake, yet, anoth, court, decis, that, de...       0
8  [mani, peopl, have, rais, the, alarm, regard, ...       0
9  [just, when, you, might, have, thought, get, b...       0


In [18]:
data['text']=data['text'].apply(' '.join)
print(data.head(10))

                                                text  target
0  donald trump just couldn wish all american hap...       0
1  hous intellig committe chairman devin nune hav...       0
2  friday was reveal that former milwauke sheriff...       0
3  christma day donald trump announc that would b...       0
4  pope franci use his annual christma day messag...       0
5  the number case cop brutal and kill peopl colo...       0
6  donald trump spent good portion his day his go...       0
7  the wake yet anoth court decis that derail don...       0
8  mani peopl have rais the alarm regard the fact...       0
9  just when you might have thought get break fro...       0


### SPLITTING DATA

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
x_train,x_test,y_train,y_test=train_test_split(data['text'],data['target'],test_size=0.25)
display(x_train.head())
print('\n')
display(y_train.head())

27467    washington reuter presid donald trump announc ...
16772    some great news the carbon emiss rule the supr...
32656    washington reuter democrat u.s. presidenti can...
2100     one the thing that make trumpcar such blow job...
29014    washington reuter presid donald trump wall alo...
Name: text, dtype: object

27467    1
16772    0
32656    1
2100     0
29014    1
Name: target, dtype: int64

### VECTORIZATION

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [23]:
my_tfidf = TfidfVectorizer( max_df=0.7)
tfidf_train= my_tfidf.fit_transform(x_train)
tfidf_test= my_tfidf.transform(x_test)

In [24]:
print(tfidf_train)

  (0, 16776)	0.04231808571501532
  (0, 73307)	0.026945259920130925
  (0, 39416)	0.04928779185943848
  (0, 73280)	0.06317083950173227
  (0, 19024)	0.0470298607787371
  (0, 51333)	0.11237733256723917
  (0, 21453)	0.05365476721480929
  (0, 10248)	0.07888118488146921
  (0, 1252)	0.07228422741404612
  (0, 12265)	0.023910218332628196
  (0, 80227)	0.048708951104590274
  (0, 41321)	0.044040276126781526
  (0, 41478)	0.03914725912167589
  (0, 63290)	0.07194820216741901
  (0, 24117)	0.04454314907537373
  (0, 80212)	0.03748777540766966
  (0, 31750)	0.03966467712470677
  (0, 52442)	0.06124517610463897
  (0, 82832)	0.052087286511363726
  (0, 73694)	0.041456383731741986
  (0, 29089)	0.02847810790650961
  (0, 85585)	0.05355913286677454
  (0, 64121)	0.027695736251291736
  (0, 32578)	0.02869735819675118
  (0, 59448)	0.040732699312302695
  :	:
  (33672, 60169)	0.01651107250524091
  (33672, 73676)	0.026822459404067665
  (33672, 41261)	0.03212717029709534
  (33672, 85801)	0.04693236976858993
  (33672, 3132

### LOGISTIC REGRESSION

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [26]:
model_1 = LogisticRegression(max_iter=900)
model_1.fit(tfidf_train, y_train)

LogisticRegression(max_iter=900)

In [27]:
# Prediction

pred_1 = model_1.predict(tfidf_test)
crl = accuracy_score(y_test,pred_1)
print(crl*100)

98.70824053452117


### PASSIVE AGGRESIVE CLASSIFIER

In [28]:
from sklearn.linear_model import PassiveAggressiveClassifier

In [29]:
model = PassiveAggressiveClassifier(max_iter=50)
model.fit(tfidf_train, y_train)

PassiveAggressiveClassifier(max_iter=50)

In [30]:
# Prediction

y_pred = model.predict(tfidf_test)
accscore = accuracy_score(y_test, y_pred)
print('The Accuracy of prediction is',accscore*100)

The Accuracy of prediction is 99.58129175946547
